In [ ]:

import pandas as pd
import numpy as np


df = pd.read_csv("cleaned_data.csv")

from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api


info = api.info()


model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import string
def preprocess(x):
  x = str(x)
  x = x.lower()
  x = "".join([char for char in x if char not in string.punctuation])
  words = word_tokenize(x)
  words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
  x = ' '.join(words)
  return x
def cat_preprocess(x):
  x = str(x)
  x = x.lower()
  x = x.split('-')
  x = ' '.join(x)
  return x

In [ ]:
df['Review']= df['Review'].apply(preprocess)
df['Review']

0                                                     good
1        app useful certain phone brand excepted phone ...
2                          always promoting antihindu show
3        im able remove app anymore 0 5 top force u rev...
4             thy supporting maharaj movie thts acceptable
                               ...                        
19995    cant even use im away university wont even let...
19996    app supposed allow download show let watch off...
19997    worst watching oneplus 9 pro locking phone who...
19998    rely password sharing since expensive account ...
19999    3 day able get phone app open although husband...
Name: Review, Length: 20000, dtype: object

In [ ]:
def rev_embed(x):  # Pass the model as an argument
    x = x.split(' ')
    embedd = []
    for i in x:
        if i in model:  # Check if word is in vocabulary
            embedd.append(model[i]) # size  of model.wv[i] is 100
    if embedd:  # If any embeddings were found
        x = np.array(embedd)
        x = np.mean(x, axis=0)
        return x
    else:
        return np.zeros(model.vector_size)  # Return a zero vector for missing words


In [ ]:
feat_vect_glove = []
for i in df['Review']:
  feat_vect_glove.append(rev_embed(i))

In [ ]:
vect_glove_hash = {}
for i in range(len(df)):
  name = str(df["Rating"][i])+ " " + df["virality"][i]
  vect_glove_hash[name] = rev_embed(df["Review"][i]).tolist()

vect_glove_hash


{'3 level 1': [0.06399223953485489,
  0.006154493894428015,
  -0.022649245336651802,
  0.04869807884097099,
  -0.05713445320725441,
  0.021985141560435295,
  0.04428967460989952,
  -0.06574735045433044,
  0.10624001175165176,
  0.035224221646785736,
  -0.04322520270943642,
  -0.11722218245267868,
  -0.0428137332201004,
  -0.052127838134765625,
  -0.024903731420636177,
  0.08450109511613846,
  0.08765064924955368,
  0.08377907425165176,
  -0.02145593799650669,
  -0.10562410950660706,
  0.044439490884542465,
  0.08622481673955917,
  0.019073486328125,
  0.019288843497633934,
  0.014490127563476562,
  0.01896216720342636,
  -0.0526450015604496,
  0.07352308928966522,
  -0.0029158159159123898,
  -0.041435934603214264,
  -0.05535750091075897,
  -0.00685258349403739,
  -0.07915730774402618,
  -0.022818826138973236,
  0.048500753939151764,
  -0.020392678678035736,
  0.016478104516863823,
  0.012202956713736057,
  0.05557112395763397,
  0.08214397728443146,
  0.07281077653169632,
  -0.07843156

In [ ]:
feat_vect_glove=np.array(feat_vect_glove)
feat_vect_glove

array([[ 0.04052734,  0.0625    , -0.01745605, ..., -0.15625   ,
         0.00193024, -0.06982422],
       [ 0.07850429, -0.05728068, -0.00197056, ..., -0.03817313,
        -0.0138716 , -0.01232038],
       [-0.02160645,  0.07023112, -0.05206299, ..., -0.08410645,
         0.08732096, -0.07291666],
       ...,
       [ 0.03757176,  0.0244996 , -0.02699973, ..., -0.05554014,
        -0.02600098, -0.01118238],
       [ 0.07707836,  0.03011477, -0.0163642 , ..., -0.0658818 ,
        -0.0372922 , -0.01720711],
       [ 0.00205994, -0.02994919, -0.02560425, ..., -0.05920346,
        -0.01087443, -0.05759684]])

In [ ]:
feat_vect_glove.shape

(20000, 300)

In [21]:
np.save('./feat_vect_glove.npy', feat_vect_glove)

In [ ]:

print(emb_dict.keys())
json_object = json.dumps(emb_dict)

with open("word2vec_0_14152.json", "w") as outfile:
    outfile.write(json_object)